# Classical Models - Training & Evaluation

This notebook trains all **Classical Baselines** for the comparative study.

**Models Trained:**
1. Logistic Regression
2. Random Forest
3. LightGBM
4. XGBoost
5. Isolation Forest (Unsupervised)
6. One-Class SVM (Unsupervised)
7. Autoencoder (Unsupervised)

**Output:** `results/tables/classical_results.csv`

In [4]:
import pandas as pd
import numpy as np
import time
import os
import sys

# Ensure project root is in path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src.data.load_data import load_config, load_creditcard_data
from src.data.preprocess import preprocess_creditcard
from src.features.dimensionality_reduction import apply_pca
from src.utils.seed import set_seed
from src.evaluation.metrics import calculate_metrics
from src.evaluation.cost_sensitive import calculate_financial_loss

# Classical Models
from src.classical.supervised_baselines import LogisticRegressionModel, RandomForestModel
from src.classical.lightgbm_model import LightGBMModel
from src.classical.xgboost_model import XGBoostModel
from src.classical.isolation_forest import IsolationForestModel
from src.classical.one_class_svm import OneClassSVMModel
from src.classical.autoencoder import AutoEncoderModel

import warnings
warnings.filterwarnings('ignore')

In [5]:
# Configuration
config_path = os.path.join(project_root, 'src', 'config', 'config.yaml')
config = load_config(config_path)
N_COMPONENTS = config['quantum']['n_qubits'] # Match Quantum Dimension for Fairness
SEED = 42

print(f"Using Feature Dimension (PCA): {N_COMPONENTS}")

Using Feature Dimension (PCA): 8


In [6]:
# Load & Process Data
df = load_creditcard_data(config)
df_clean = preprocess_creditcard(df, config)

X_all = df_clean.drop(columns=['Class']).values
y_all = df_clean['Class'].values

# Apply PCA
X_pca, _ = apply_pca(X_all, n_components=N_COMPONENTS)

# Split (Reproducible)
from sklearn.model_selection import train_test_split

# Subsample for speed if needed, but classical can handle full data.
# However, to be EXACTLY comparable to Quantum (which needs subsampling),
# we should ideally use the same subsample. 
# BUT, the user prompt said "Train ALL classical models... Do NOT compare here".
# Usually classical is trained on full. Let's train on FULL for now to show Classical 'at its best' 
# or subsample to match Quantum exactly? 
# The user said "Use identical ... splits". 
# If Quantum is restricted to 1000 samples, Classical should probably also be restricted IF we want strict fairness.
# But usually Classical advantage IS scalability. 
# Let's use a large subset (e.g. 10k or full) here, and in the COMPARISON notebook we strictly align.
# Actually, to save time in this notebook execution, let's use a reasonable subset (e.g. 10k).

# Let's use 20% stratified test set, consistent with main pipeline
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_pca, y_all, test_size=0.2, stratify=y_all, random_state=SEED
)

# For this notebook, we verify classical models work. We'll save results on this split.
print(f"Train Shape: {X_train_full.shape}, Test Shape: {X_test.shape}")

2026-01-28 12:00:47,482 INFO Loading Credit Card data from c:\Users\mostr\OneDrive\Documents\GitHub\Anomaly-detection-in-finance-using-QML\data/raw\creditcard.csv
2026-01-28 12:00:49,328 INFO Loaded Credit Card data with shape (284807, 31)
2026-01-28 12:00:50,695 INFO Cleaned data: (284807, 31) -> (283726, 31)
2026-01-28 12:00:50,865 INFO Normalized 29 columns using minmax
2026-01-28 12:00:51,041 INFO PCA reduced to 8 components. Explained Variance: 1.0000
Train Shape: (226980, 8), Test Shape: (56746, 8)


In [7]:
def train_evaluate(model, name, X_train, y_train, X_test, y_test):
    print(f"Training {name}...")
    start = time.time()
    
    # Fit
    if name in ['Isolation Forest', 'One-Class SVM']:
        model.fit(X_train)
    elif name == 'Autoencoder':
        model.fit(X_train)
    else:
        model.fit(X_train, y_train)
        
    train_time = time.time() - start
    
    # Predict
    start_inf = time.time()
    if hasattr(model, 'predict_proba'):
        scores = model.predict_proba(X_test)
        if scores.ndim > 1: scores = scores[:, 1]
    elif hasattr(model, 'score'):
        # IsoForest/OCSVM: higher score = normal. Invert for anomaly score.
        scores = -model.score(X_test)
    elif hasattr(model, 'predict_anomaly_score'):
        scores = model.predict_anomaly_score(X_test)
    else:
        scores = model.predict(X_test)
    inf_time = time.time() - start_inf
    
    # Calculate Logic for Threshold-Independent Metrics
    from sklearn.metrics import average_precision_score, roc_auc_score
    rc = roc_auc_score(y_test, scores)
    pr = average_precision_score(y_test, scores)
    
    return {
        "model": name,
        "type": "classical",
        "roc_auc": rc,
        "pr_auc": pr,
        "train_time": train_time,
        "inference_time": inf_time
    }

In [8]:
models = [
    ("Logistic Regression", LogisticRegressionModel(random_state=SEED)),
    ("Random Forest", RandomForestModel(random_state=SEED)),
    ("LightGBM", LightGBMModel(random_state=SEED)),
    ("XGBoost", XGBoostModel(random_state=SEED)),
    ("Isolation Forest", IsolationForestModel(random_state=SEED)),
    ("One-Class SVM", OneClassSVMModel()),
    ("Autoencoder", AutoEncoderModel(input_dim=N_COMPONENTS, epochs=10))
]

results = []
for name, model in models:
    try:
        res = train_evaluate(model, name, X_train_full, y_train_full, X_test, y_test)
        results.append(res)
        print(f"   -> PR-AUC: {res['pr_auc']:.4f}")
    except Exception as e:
        print(f"   Failed {name}: {e}")

Training Logistic Regression...
2026-01-28 12:00:53,741 INFO Training Logistic Regression on shape (226980, 8)
   -> PR-AUC: 0.2822
Training Random Forest...
2026-01-28 12:01:06,050 INFO Training Random Forest on shape (226980, 8)
   -> PR-AUC: 0.5645
Training LightGBM...
2026-01-28 12:01:13,531 INFO Training LightGBM on shape (226980, 8)
   -> PR-AUC: 0.0118
Training XGBoost...
2026-01-28 12:01:14,102 INFO Training XGBoost on shape (226980, 8)
   -> PR-AUC: 0.5770
Training Isolation Forest...
2026-01-28 12:01:14,610 INFO Training Isolation Forest on shape (226980, 8)
   -> PR-AUC: 0.0532
Training One-Class SVM...
2026-01-28 12:01:15,417 INFO Training One-Class SVM on shape (226980, 8)
   -> PR-AUC: 0.0020
Training Autoencoder...
2026-01-28 12:28:49,626 INFO Training Autoencoder on (226980, 8) using cpu
2026-01-28 12:30:15,423 INFO Epoch [10/10], Loss: 281886250.8035
   -> PR-AUC: 0.0019


In [9]:
# Save Results
df_res = pd.DataFrame(results)
os.makedirs(os.path.join(project_root, 'results', 'tables'), exist_ok=True)
df_res.to_csv(os.path.join(project_root, 'results', 'tables', 'classical_results.csv'), index=False)
df_res

,model,type,roc_auc,pr_auc,train_time,inference_time
0,Logistic Regression,classical,0.940249,0.282195,12.247998,0.008858
1,Random Forest,classical,0.910013,0.564512,7.396258,0.074539
2,LightGBM,classical,0.826695,0.011784,0.507650,0.040842
3,XGBoost,classical,0.934130,0.577013,0.467108,0.017924
4,Isolation Forest,classical,0.868704,0.053192,0.351552,0.425316
5,One-Class SVM,classical,0.551936,0.002004,1541.346205,112.814303
6,Autoencoder,classical,0.459921,0.001865,85.811203,0.027728
